In [3]:
from hyperopt import hp,tpe,fmin
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
# Load dataset
mnist = input_data.read_data_sets('./dataset/MNIST_data',one_hot=True)

Extracting ./dataset/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./dataset/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./dataset/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./dataset/MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
batch_size = 2000                                  #单批次数据量
hidden_layer1_node = 500                         #隐藏层1节点数
hidden_layer2_node = 300                         #隐藏层2节点数
output_layer_node = 10                            #输出层节点数
steps = 500                                        #训练迭代次数
lr = 0.3                                          #学习速率
regularization_rate = 0.0015                       #正则化系数
#image_num = 300

In [14]:
def nn_train(batch_size,hidden_layer1_node,hidden_layer2_node,output_layer_node,steps,lr,regularization_rate):
    from hyperopt import hp,tpe,fmin
    import tensorflow as tf
    import numpy as np
    from tensorflow.examples.tutorials.mnist import input_data
    
    # Load dataset
    mnist = input_data.read_data_sets('./dataset/MNIST_data',one_hot=True)
    
    #插入点
    x = tf.placeholder(tf.float32,[None,784])
    y = tf.placeholder(tf.float32,[None,output_layer_node])
    keep_prob = tf.placeholder(tf.float32)
    
    #变量
    w1 = tf.Variable(tf.truncated_normal(shape=[784,hidden_layer1_node],stddev=0.1))
    w2 = tf.Variable(tf.truncated_normal(shape=[hidden_layer1_node,hidden_layer2_node],stddev=0.1))
    w3 = tf.Variable(tf.truncated_normal(shape=[hidden_layer2_node,output_layer_node],stddev=0.1))
    b1 = tf.Variable(tf.constant(0.1,shape=[hidden_layer1_node]),name='b1')
    b2 = tf.Variable(tf.constant(0.1,shape=[hidden_layer2_node]),name='b2')
    b3 = tf.Variable(tf.constant(0.1,shape=[output_layer_node]),name='b3')
    
    #神经网络结构
    hidden_layer1 = tf.matmul(x,w1) + b1
    hidden_layer1 = tf.nn.relu(hidden_layer1)
    hidden_layer1 = tf.nn.dropout(x=hidden_layer1,keep_prob=keep_prob)
    
    hidden_layer2 = tf.matmul(hidden_layer1,w2) + b2
    hidden_layer2 = tf.nn.relu(hidden_layer2)
    hidden_layer2 = tf.nn.dropout(x=hidden_layer2,keep_prob=keep_prob)
    
    out_layer = tf.matmul(hidden_layer2,w3) + b3
    
    #正则化
    regular = tf.contrib.layers.l2_regularizer(regularization_rate)
    regularization = regular(w1) + regular(w2) + regular(w3)
    
    #Cost Function
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=out_layer,labels=y)) + regularization
    
    #optimi
    train_step = tf.train.AdadeltaOptimizer(lr).minimize(loss)
    
    #accuracy cal
    correct_prediction = tf.equal(tf.argmax(out_layer),tf.argmax(y))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    
    #session
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        for i in range(steps):
            x_train,y_train = mnist.train.next_batch(i)
            sess.run(train_step,feed_dict={x:x_train,y:y_train,keep_prob:1.0})
            if i % 50 == 0:
                acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0}) 
                print('Iter %d, Testing Accuracy: %f' % (i+1,acc))
        return sess.run(loss,feed_dict={x:mnist.train.images,y:mnist.train.labels,keep_prob:1.0})

In [15]:
nn_train(2000,hidden_layer1_node,hidden_layer2_node,output_layer_node,steps,lr,regularization_rate)

Extracting ./dataset/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./dataset/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./dataset/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./dataset/MNIST_data/t10k-labels-idx1-ubyte.gz
Iter 1, Testing Accuracy: 0.000000
Iter 51, Testing Accuracy: 0.000000
Iter 101, Testing Accuracy: 0.000000
Iter 151, Testing Accuracy: 0.000000
Iter 201, Testing Accuracy: 0.000000
Iter 251, Testing Accuracy: 0.000000
Iter 301, Testing Accuracy: 0.000000
Iter 351, Testing Accuracy: 0.000000
Iter 401, Testing Accuracy: 0.000000
Iter 451, Testing Accuracy: 0.000000


2.886658